In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Assignment/DataScienceCaseStudy/dataSample.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 11 columns):
Unnamed: 0               16162 non-null int64
ID                       16162 non-null int64
Type                     16162 non-null object
event_timestamp          16162 non-null int64
event_label              16162 non-null object
log_timestamp            16162 non-null int64
log_reading_1            15848 non-null float64
log_reading_2            15883 non-null object
log_reading_3            15843 non-null float64
log_reading_4            16162 non-null int64
warranty_service_flag    1389 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


# Preprocessing

In [3]:
df=pd.get_dummies(df)
df=df.rename(columns={'Unnamed: 0':'sl_number'})
df.columns

Index(['sl_number', 'ID', 'event_timestamp', 'log_timestamp', 'log_reading_1',
       'log_reading_3', 'log_reading_4', 'Type_TypeA', 'Type_TypeB',
       'event_label_activate', 'event_label_break', 'log_reading_2_a',
       'log_reading_2_b', 'log_reading_2_c', 'warranty_service_flag_n',
       'warranty_service_flag_y'],
      dtype='object')

In [4]:
df=df.drop(columns=['event_label_activate','warranty_service_flag_n'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 14 columns):
sl_number                  16162 non-null int64
ID                         16162 non-null int64
event_timestamp            16162 non-null int64
log_timestamp              16162 non-null int64
log_reading_1              15848 non-null float64
log_reading_3              15843 non-null float64
log_reading_4              16162 non-null int64
Type_TypeA                 16162 non-null uint8
Type_TypeB                 16162 non-null uint8
event_label_break          16162 non-null uint8
log_reading_2_a            16162 non-null uint8
log_reading_2_b            16162 non-null uint8
log_reading_2_c            16162 non-null uint8
warranty_service_flag_y    16162 non-null uint8
dtypes: float64(2), int64(5), uint8(7)
memory usage: 994.5 KB


In [5]:
df['log_reading_1'].fillna(df['log_reading_1'].median(),inplace=True)
df['log_reading_3'].fillna(df['log_reading_3'].median(),inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 14 columns):
sl_number                  16162 non-null int64
ID                         16162 non-null int64
event_timestamp            16162 non-null int64
log_timestamp              16162 non-null int64
log_reading_1              16162 non-null float64
log_reading_3              16162 non-null float64
log_reading_4              16162 non-null int64
Type_TypeA                 16162 non-null uint8
Type_TypeB                 16162 non-null uint8
event_label_break          16162 non-null uint8
log_reading_2_a            16162 non-null uint8
log_reading_2_b            16162 non-null uint8
log_reading_2_c            16162 non-null uint8
warranty_service_flag_y    16162 non-null uint8
dtypes: float64(2), int64(5), uint8(7)
memory usage: 994.5 KB


# Train test split

In [6]:
x=df.drop(columns=['event_label_break'],axis=1)
y=df['event_label_break']
print(f'The data has {y.mean()*100:.2f}% of positive points')

The data has 34.15% of positive points


In [7]:
from sklearn.model_selection import train_test_split
x_tr,x_te,y_tr,y_te=train_test_split(x,y,stratify=y,test_size=.27,random_state=10)
print(x_tr.shape,x_te.shape,y_tr.shape,y_te.shape)
print(f'Training set has {y_tr.mean()*100:.2f}% positive points and test sets {y_te.mean()*100:.2f}% of positive points')

(11798, 13) (4364, 13) (11798,) (4364,)
Training set has 34.15% positive points and test sets 34.14% of positive points


# XGboost

In [8]:
from xgboost import XGBClassifier

C:\Users\udipt\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


In [9]:
xg=XGBClassifier(n_jobs=-1,random_state=10)
xg.fit(x_tr,y_tr)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=10, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
y1_pred=xg.predict(x_te)
y1_prob=xg.predict_proba(x_te)
print(f'XGboost without grid search predicted {y1_pred.mean()*100:.2f}% points as positive in test data')

XGboost without grid search predicted 37.69% points as positive in test data


## Grid search


In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
pg={'n_estimators':range(250,300,10)}
xg_cv=GridSearchCV(xg,pg,cv=5,scoring='roc_auc',n_jobs=-1)
xg_cv.fit(x_tr,y_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_esti...rs=100, n_jobs=-1,
                                     num_parallel_tree=1,
                                     objective='binary:logistic',
                                     random_state=10, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, sub

In [13]:
print(f'Best parameters are {xg_cv.best_params_} with best score {xg_cv.best_score_}')

Best parameters are {'n_estimators': 250} with best score 0.7720336251988795


In [14]:
xg_best=xg_cv.best_estimator_

In [15]:
yxg_pred=xg_best.predict(x_te)
yxg_prob=xg_best.predict_proba(x_te)
print(f'Grid searched model predicted {yxg_pred.mean()*100:.2f}% points as positive')

Grid searched model predicted 35.45% points as positive


# Lift

In [16]:
predicted_prob=[]
for i in yxg_prob:
    predicted_prob.append(i[1])
predicted_prob[21:22]

[0.6971924]

In [17]:
yxg_prob[21:22]

array([[0.30280763, 0.6971924 ]], dtype=float32)

In [18]:
yxg_pred[1:10]

array([0, 1, 0, 0, 1, 0, 0, 0, 0], dtype=uint8)

In [19]:
comp=pd.DataFrame(predicted_prob)
comp.head()

,0
0,0.268436
1,0.001291
2,0.704080
3,0.281567
4,0.011931


In [22]:
test_set=y_te.reset_index()
test_set

,index,event_label_break
0,13877,1
1,6815,1
2,9851,0
3,16012,1
4,130,0
...,...,...
4359,3930,0
4360,7891,0
4361,12435,1
4362,8176,1


In [23]:
test_set=test_set.drop(columns='index',axis=1)
test_set

,event_label_break
0,1
1,1
2,0
3,1
4,0
...,...
4359,0
4360,0
4361,1
4362,1


In [24]:
comp['actual_values']=test_set
comp.columns

Index([0, 'actual_values'], dtype='object')

In [25]:
comp=comp.rename(columns={0:'predicted_probabilities'})
comp.head()

,predicted_probabilities,actual_values
0,0.268436,1
1,0.001291,1
2,0.704080,0
3,0.281567,1
4,0.011931,0


In [26]:
comp['predicted_probabilities'].mean()

0.33542467397144987

In [27]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 2 columns):
predicted_probabilities    4364 non-null float64
actual_values              4364 non-null uint8
dtypes: float64(1), uint8(1)
memory usage: 38.5 KB


In [28]:
comp.sort_values('predicted_probabilities', ascending=False)

,predicted_probabilities,actual_values
1866,0.998291,1
1420,0.997538,1
634,0.997482,0
278,0.997222,1
1798,0.996950,1
...,...,...
3036,0.000302,0
503,0.000221,0
2241,0.000165,0
177,0.000134,0


In [91]:
comp['decile_rank']=pd.qcut(comp['predicted_probabilities'],10,labels=False)
comp.head()

,predicted_probabilities,actual_values,decile_rank
1866,0.998291,1,9
1420,0.997538,1,9
634,0.997482,0,9
278,0.997222,1,9
1798,0.996950,1,9


In [92]:
comp['decile_rank'].value_counts()

3    437
9    437
5    437
0    437
7    436
6    436
2    436
1    436
8    436
4    436
Name: decile_rank, dtype: int64

In [93]:
comp=comp.sort_values('predicted_probabilities',ascending=False)
comp.head()

,predicted_probabilities,actual_values,decile_rank
1866,0.998291,1,9
1420,0.997538,1,9
634,0.997482,0,9
278,0.997222,1,9
1798,0.996950,1,9


In [94]:
top_decile=comp[comp['decile_rank']==9]
top_decile.head()

,predicted_probabilities,actual_values,decile_rank
1866,0.998291,1,9
1420,0.997538,1,9
634,0.997482,0,9
278,0.997222,1,9
1798,0.996950,1,9


In [87]:
comp.to_csv('C:/Users/udipt/Desktop/Websoft/Performance/Comparison.csv',header=1)

In [95]:
top_decile['predicted_probabilities'].mean()

0.8703211570495326

In [96]:
lift=top_decile['predicted_probabilities'].mean()/comp['predicted_probabilities'].mean()
print(f'lift = {lift:.2f}')

lift = 2.59


In [62]:
dec_8=comp[comp['decile_rank']==8]
dec_7=comp[comp['decile_rank']==7]
dec_6=comp[comp['decile_rank']==6]
dec_5=comp[comp['decile_rank']==5]
dec_4=comp[comp['decile_rank']==4]
dec_3=comp[comp['decile_rank']==3]
dec_2=comp[comp['decile_rank']==2]
dec_1=comp[comp['decile_rank']==1]
dec_0=comp[comp['decile_rank']==0]

In [97]:
dec_8['predicted_probabilities'].mean()/comp['predicted_probabilities'].mean()

2.1743316148499265

In [98]:
comp.groupby('decile_rank')[['predicted_probabilities','actual_values']].mean()

,predicted_probabilities,actual_values
decile_rank,,
0,0.002029,0.009153
1,0.005195,0.011468
2,0.012468,0.027523
3,0.042550,0.116705
4,0.191407,0.458716
5,0.372335,0.613272
6,0.505222,0.594037
7,0.623521,0.548165
8,0.729324,0.497706


In [100]:
x_te.columns

Index(['sl_number', 'ID', 'event_timestamp', 'log_timestamp', 'log_reading_1',
       'log_reading_3', 'log_reading_4', 'Type_TypeA', 'Type_TypeB',
       'log_reading_2_a', 'log_reading_2_b', 'log_reading_2_c',
       'warranty_service_flag_y'],
      dtype='object')

In [102]:
x_te.head()

,sl_number,ID,event_timestamp,log_timestamp,log_reading_1,log_reading_3,log_reading_4,Type_TypeA,Type_TypeB,log_reading_2_a,log_reading_2_b,log_reading_2_c,warranty_service_flag_y
13877,13877,549783,1594922787,1437030415,10.0,10.0,6,0,1,1,0,0,0
6815,6815,95321,1359908656,1434251185,10.0,10.0,6,0,1,1,0,0,0
9851,9851,724307,1473380907,1435461182,10.0,6.0,8,1,0,1,0,0,0
16012,16012,702211,1441476449,1437718017,10.0,10.0,7,1,0,1,0,0,0
130,130,709365,1431615901,1432285540,10.0,10.0,10,1,0,1,0,0,0


In [116]:
test_variables=x_te.reset_index()
test_variables=test_variables.drop(columns='index',axis=1)
test_variables.head()

,sl_number,ID,event_timestamp,log_timestamp,log_reading_1,log_reading_3,log_reading_4,Type_TypeA,Type_TypeB,log_reading_2_a,log_reading_2_b,log_reading_2_c,warranty_service_flag_y
0,13877,549783,1594922787,1437030415,10.0,10.0,6,0,1,1,0,0,0
1,6815,95321,1359908656,1434251185,10.0,10.0,6,0,1,1,0,0,0
2,9851,724307,1473380907,1435461182,10.0,6.0,8,1,0,1,0,0,0
3,16012,702211,1441476449,1437718017,10.0,10.0,7,1,0,1,0,0,0
4,130,709365,1431615901,1432285540,10.0,10.0,10,1,0,1,0,0,0


In [122]:
test_variables.to_csv('C:/Users/udipt/Desktop/Websoft/Performance/Independent_vars.csv',header=1)

In [138]:
t_vars=pd.DataFrame(test_variables)

In [148]:
final_df=pd.concat([t_vars,comp],axis=1)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4364 entries, 0 to 4363
Data columns (total 29 columns):
sl_number                  4364 non-null int64
ID                         4364 non-null int64
event_timestamp            4364 non-null int64
log_timestamp              4364 non-null int64
log_reading_1              4364 non-null float64
log_reading_3              4364 non-null float64
log_reading_4              4364 non-null int64
Type_TypeA                 4364 non-null uint8
Type_TypeB                 4364 non-null uint8
log_reading_2_a            4364 non-null uint8
log_reading_2_b            4364 non-null uint8
log_reading_2_c            4364 non-null uint8
warranty_service_flag_y    4364 non-null uint8
predicted_probabilities    4364 non-null float64
actual_values              4364 non-null uint8
decile_rank                4364 non-null int64
sl_number                  4364 non-null object
ID                         4364 non-null object
event_timestamp            4364 non-nul